In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import sqlite3 as db

In [2]:
#https://nofluffjobs.com
def Generowanie_linku(lokalizacja,technologia,doswiadczenie):
    link = f'https://nofluffjobs.com/pl/{lokalizacja}/{technologia}?criteria=seniority%3D{doswiadczenie}'
    return link   
    

strona=Generowanie_linku('krakow','Python','expert')
print(strona)


https://nofluffjobs.com/pl/krakow/Python?criteria=seniority%3Dexpert


In [3]:
lokalizacje = ['warszawa','poznan','krakow','wroclaw','gdansk','trojmiasto','katowice','slask','lodz','bialystok',
               'gdynia','lublin','rzeszow','bydgoszcz','gliwice','czestochowa','szczecin','sopot','kielce','torun']

technologie = ['Java','Python','C%23','SQL','C%2B%2B','Golang','JavaScript','React','Angular','TypeScript','HTML']

doswiadczenia = ['trainee','junior','mid','senior','expert']

In [4]:
do_linkow = []
for item_t in technologie:
    for item_d in doswiadczenia:
        for item_l in lokalizacje:
            lista = [item_t,item_d,item_l]
            do_linkow.append(lista)

In [5]:
len(do_linkow)

1100

In [6]:
lista_linkow = []
for i in do_linkow:
    link = Generowanie_linku(i[2],i[0],i[1])
    lista_linkow.append(link) 

In [7]:
len(lista_linkow)

1100

In [8]:
def Zbieranie_danych_z_linku(link):
    resp = requests.get(link)
    page = resp.text
    soup = BeautifulSoup(page, 'html.parser')
    posting_list = soup.find_all('nfj-postings-list')
    name_list = []
    salary_list = []
    for item_p_l in posting_list:
        if item_p_l.get('listname') == 'search':
            oferts = item_p_l.find_all('aside')
            for i in range(0,len(oferts),2):
                for j in oferts[i].find_all('h3'):
                    if j.get('data-cy') == 'title position on the job offer listing':
                        name = j.text.split('NOWA')[0]
                        name_list.append(name)
                for j in  oferts[i].find_all('span'):
                    if j.get('data-cy') == "salary ranges on the job offer listing":
                        salary = j.text.replace('\xa0','').replace(' ','').replace('PLN\n','')
                        salary_list.append(salary)
    return name_list, salary_list

In [9]:
nazwy = []
wyplaty = []
technologia = []
doswiadczenie = []
lokalizacja = []
x = 0
for link1 in lista_linkow:
    dane1 = Zbieranie_danych_z_linku(link1)
    nazwy_z_linku = dane1[0]
    wyplaty_z_linku = dane1[1]
    for nazwa in nazwy_z_linku:
        nazwy.append(nazwa)
    for wyplata in wyplaty_z_linku:
        wyplaty.append(wyplata)
        technologia.append(do_linkow[x][0])
        doswiadczenie.append(do_linkow[x][1])
        lokalizacja.append(do_linkow[x][2])
    x += 1

In [10]:
dane = {'Nazwa':nazwy,'Technologia':technologia,'Doświadczenie':doswiadczenie,'Lokalizacja':lokalizacja,'Wypłata':wyplaty}
df = pd.DataFrame(dane)

In [11]:
df

,Nazwa,Technologia,Doświadczenie,Lokalizacja,Wypłata
0,Trainee Java Developer,Java,trainee,warszawa,bezpłatnystaż\n
1,Projektowanie / ilustracja / grafika,Java,trainee,warszawa,2800–5000
2,Projektowanie / ilustracja / grafika,Java,trainee,warszawa,2800–5000
3,Projektowanie / ilustracja / grafika,Java,trainee,krakow,2800–5000
4,Projektowanie / ilustracja / grafika,Java,trainee,krakow,2800–5000
...,...,...,...,...,...
2714,Senior Software Developer,HTML,senior,szczecin,18480–30240
2715,Head of SEO,HTML,expert,warszawa,12000–14000
2716,Product Designer (UX/UI),HTML,expert,poznan,18000–28000
2717,Senior Full-stack Developer (Angular/C++),HTML,expert,krakow,18000–23000


In [13]:
conn = db.connect('IT_Wynagrodzenia')
df.to_sql('IT_praca_wynagrodzenie', conn, if_exists='replace', index=True)

2719

In [14]:
conn.close()